In [1]:
# Dependencies
import requests
import bs4
from datetime import datetime
from pytz import timezone
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
# Make a reference to the books.csv file path
csv_path = "Resources/180826_Newegg_ProductId.csv"

# Import the books.csv file as a DataFrame
newegg_productid_df = pd.read_csv(csv_path, encoding="utf-8")
newegg_productid_df

,Product,productid
0,XA2_Black,N82E16875212409
1,XA2_Silver,N82E16875212412
2,XA2_Pink,9SIA29P73V2368
3,XA2_Blue,9SIA25V70A2188
4,XA2_Ultra_Black,N82E16875212413
5,XA2_Ultra_Silver,N82E16875212416
6,XA2_Ultra_Blue,9SIA29P74X2422
7,L2_Black,9SIA29P6XF7954
8,L2_Pink,9SIA29P6XF7956
9,L2_Gold,9SIA29P6XF7957


In [3]:
newegg_productid_df["productid"] = newegg_productid_df["productid"].astype(object)

In [4]:
newegg_product_id_list = newegg_productid_df["productid"]

In [5]:
master_df = None

In [6]:
for productid in newegg_product_id_list:
    
    options = Options()

    options.set_headless(True)

    driver = webdriver.Chrome(executable_path='chromedriver.exe')

    driver.get(f"https://www.newegg.com/Product/Product.aspx?Item={productid}")
    
    time.sleep(1)

    html = driver.page_source.encode('utf-8')

    soup = bs4.BeautifulSoup(html, "html.parser")
    
    # Making empty lists
    seller_list = []
    price_list = []
    
    try:     
        seller = soup.select(".featured-seller")[0].getText()[21:]
        if seller.find("Sold and Shipped by") > 0:
            seller = seller[:seller.find("Sold and Shipped by")]
            seller = seller + " in Newegg.com"
            seller_list.append(seller)
        else:
            seller = seller + " in Newegg.com"
            seller_list.append(seller)
    except:
        seller = seller + " in Newegg.com"
        seller_list.append(seller)
   
        
    if soup.select(".price-current.hidden") != []:
        price = soup.select(".price-current.hidden")[0].getText()
        price = price.replace("$", "")
        price = float(price)
        price_list.append(price)
    
    elif soup.select("span.price-current") == []:
        price = ""
        price_list.append(price)    
    
    else:
        price = soup.select("span.price-current")[0].getText()
        price = price.replace("$", "")
        price = float(price)
        price_list.append(price)

    product_price_df = pd.DataFrame({"Seller": seller_list,
                                     productid: price_list})
    
    product_price_df = product_price_df.set_index("Seller")
    
    master_df = pd.concat([master_df, product_price_df], axis=1, sort=False)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use setter for headless property instead of set_headless
  import sys


In [7]:
master_df = master_df.T
master_df = master_df.reset_index()
master_df = master_df.rename(columns={"index": "productid"})
master_df["productid"] = master_df["productid"].astype(object)
master_df = pd.merge(newegg_productid_df, master_df, on="productid", how="left")
master_df

,Product,productid,Focus Camera in Newegg.com,Action Packaged Inc. in Newegg.com,Breed in Newegg.com,Newegg in Newegg.com,Beach Camera in Newegg.com
0,XA2_Black,N82E16875212409,299.99,NaN,NaN,NaN,NaN
1,XA2_Silver,N82E16875212412,299.99,NaN,NaN,NaN,NaN
2,XA2_Pink,9SIA29P73V2368,299.99,NaN,NaN,NaN,NaN
3,XA2_Blue,9SIA25V70A2188,299.99,NaN,NaN,NaN,NaN
4,XA2_Ultra_Black,N82E16875212413,249.99,NaN,NaN,NaN,NaN
5,XA2_Ultra_Silver,N82E16875212416,NaN,359.95,NaN,NaN,NaN
6,XA2_Ultra_Blue,9SIA29P74X2422,NaN,359.95,NaN,NaN,NaN
7,L2_Black,9SIA29P6XF7954,199.99,NaN,NaN,NaN,NaN
8,L2_Pink,9SIA29P6XF7956,NaN,NaN,199.95,NaN,NaN
9,L2_Gold,9SIA29P6XF7957,NaN,NaN,NaN,189.99,NaN


In [8]:
newegg_result_df = master_df.drop(columns=["productid"])
newegg_result_df

,Product,Focus Camera in Newegg.com,Action Packaged Inc. in Newegg.com,Breed in Newegg.com,Newegg in Newegg.com,Beach Camera in Newegg.com
0,XA2_Black,299.99,NaN,NaN,NaN,NaN
1,XA2_Silver,299.99,NaN,NaN,NaN,NaN
2,XA2_Pink,299.99,NaN,NaN,NaN,NaN
3,XA2_Blue,299.99,NaN,NaN,NaN,NaN
4,XA2_Ultra_Black,249.99,NaN,NaN,NaN,NaN
5,XA2_Ultra_Silver,NaN,359.95,NaN,NaN,NaN
6,XA2_Ultra_Blue,NaN,359.95,NaN,NaN,NaN
7,L2_Black,199.99,NaN,NaN,NaN,NaN
8,L2_Pink,NaN,NaN,199.95,NaN,NaN
9,L2_Gold,NaN,NaN,NaN,189.99,NaN


In [9]:
newegg_result_df.to_csv("Output/newegg_result.csv", index=False)